# Idealista API experiments

🏠 Notebook para probar la API de Idealista. El acceso a la API se solicita aquí: https://developers.idealista.com/access-request

🤖 Código generado con IAs y con humanos.

In [ ]:
# API_KEY y API SECRET se solicitan aquí: https://developers.idealista.com/access-request

API_KEY = '<completa esto>'
API_SECRET = '<completa esto>'

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import time
import json
from datetime import datetime

In [ ]:
def show_inmuebles(inmuebles):
    if inmuebles:

        # Ordenar por precio ascendente
        inmuebles_ordenados = sorted(
            inmuebles,
            key=lambda x: x.get("price", float("inf"))  # Si no tiene precio, lo manda al final
        )

        print(f"✅ Inmuebles encontrados: {len(inmuebles_ordenados)}")
        for i, inmueble in enumerate(inmuebles_ordenados, 1):
            titulo = inmueble.get("suggestedTexts", {}).get("title", "Sin título")
            subtitulo = inmueble.get("suggestedTexts", {}).get("subtitle", "")
            descripcion = inmueble.get("description") or "Sin descripción"
            precio = inmueble.get("price", "Desconocido")
            url = inmueble.get("url", "No disponible")
            size = inmueble.get("size", None)
            superficie = f"{size} m²" if size else "Superficie no disponible"

            imagen_url = inmueble.get("thumbnail", "Sin imagen")

            print(f"\n🏠 Inmueble {i}")
            print(f"Título: {titulo}")
            if subtitulo:
                print(f"Zona: {subtitulo}")
            print(f"Superficie: {superficie}")
            print(f"Descripción: {descripcion[:200]}...")
            print(f"Precio: {precio} €")
            print(f"URL: {url}")
            print(f"Imagen: {imagen_url}")


    else:
        print("ℹ️ No se encontraron inmuebles.")

In [ ]:
def obtener_token_idealista():
    url = "https://api.idealista.com/oauth/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = {"grant_type": "client_credentials", "scope": "read"}

    try:
        response = requests.post(
            url,
            headers=headers,
            data=data,
            auth=HTTPBasicAuth(API_KEY, API_SECRET)
        )
        response.raise_for_status()
        token = response.json()["access_token"]
        return token
    except Exception as e:
        print(f"❌ Error al obtener token: {e}")
        return None

In [ ]:
def buscar_inmuebles_akupados(token, limite_paginas=5):
    url = "https://api.idealista.com/3.5/es/search"
    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/x-www-form-urlencoded"
    }

    palabras_clave = ["okupa", "ocupado", "sin posesión"]
    exclusiones = ["no está okupado", "no okupado", "no está ocupado", "no ocupado"]

    inmuebles_filtrados = []
    todos_los_inmuebles = []
    num_page = 1
    max_items_per_page = 50  # 50 es el máximo permitido por Idealista

    while num_page <= limite_paginas:
        payload = {
            "operation": "sale",
            "propertyType": "homes",
            "center": "40.416775,-3.703790",  # Madrid centro
            "distance": 5000, # distancia en metros
            "maxItems": max_items_per_page,
            "numPage": num_page,
            "language": "es",
            "country": "es",
            "order": "distance",
            "sort": "desc",
            # "bankOffer": "true"  # <- Para buscar inmuebles de bancos
        }

        try:
            response = requests.post(url, headers=headers, data=payload)
            response.raise_for_status()
            data = response.json()

            elementos = data.get("elementList", [])
            total = data.get("total", 0)

            todos_los_inmuebles.extend(elementos)

            for inmueble in elementos:
                texto = (inmueble.get("description", "") + " " + inmueble.get("suggestedTexts", {}).get("title", "")).lower()
                if any(p in texto for p in palabras_clave) and not any(e in texto for e in exclusiones):
                    inmuebles_filtrados.append(inmueble)

            print(f"✅ Página {num_page} procesada. Filtrados: {len(inmuebles_filtrados)} / Totales: {len(todos_los_inmuebles)}")

            if num_page * max_items_per_page >= total:
                break

            time.sleep(1)
            
            num_page += 1
        except requests.exceptions.HTTPError as e:
            print(f"❌ Error HTTP: {response.status_code} - {response.text}")
            break
        except Exception as e:
            print(f"❌ Error general: {e}")
            break

    return inmuebles_filtrados, todos_los_inmuebles

In [ ]:

def guardar_inmuebles(filename, inmuebles):
    if inmuebles:
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(inmuebles, f, ensure_ascii=False, indent=2)

        print(f"📁 Archivo '{filename}' guardado con éxito.")
    else:
        print('❌ Error, no hay inmuebles para guardar.')

In [ ]:
MAX_PAGES = 20 # ten en cuenta las condiciones de uso de la API. Puede que tengas un límite de petiones mensual o un coste por cada petición.

token = obtener_token_idealista()
if token:
    inmuebles_okupados , todos_los_inmuebles = buscar_inmuebles_akupados(token, MAX_PAGES)

    show_inmuebles(inmuebles_okupados)

In [ ]:
now = datetime.now().strftime("%Y%m%d_%H%M")

filename = f"inmuebles/inmuebles_okupados_{now}.json"
guardar_inmuebles(filename, inmuebles_okupados)

filename = f"inmuebles/todos_los_inmuebles_{now}.json"
guardar_inmuebles(filename, todos_los_inmuebles)